# **Decision Tree for Predicting Article Section Based on Headlines and Abstracts**

### **DS 5110: Big Data Systems**

### **Spring 2022**

### **Group 1 - New York Times Articles Analysis**

### **Avni Malek (am7bf), Brittany Nguyen (bhn4bf), Cecily Wolfe (cew4pf)**

## **Getting the Data**

### **Set up Directory**

In [10]:
import os
os.getcwd()

'/project/ds5559/group1_nyt'

In [11]:
os.chdir('/project/ds5559/group1_nyt')

In [12]:
os.getcwd()

'/project/ds5559/group1_nyt'

In [13]:
# use to increase driver memory to prevent Java heap space overflow
memory = '20g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [14]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark = SparkSession.builder \
        .appName("test_11") \
        .config("spark.executor.memory", '20g') \
        .getOrCreate()

### **Read in the Preprocessed Data**

In [15]:
df = spark.read.option("header", True).csv('preprocessed_data')

In [16]:
cat_cols = [field for (field, dataType) in df.dtypes if dataType == 'string']

In [17]:
df = df.fillna("", subset = cat_cols)

## **Predict Article Section Based on Abstract Using a Decision Tree Model**

In [18]:
# load modules

from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler 

from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StringIndexerModel

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer

In [19]:
# load in functions that compute top k accuracy for classification and balance the data with random under/oversampling
from ds5110_proj_funcs import top_k_accuracy, balance_categories

In [20]:
# create transformers and estimator to add to pipeline

# string indexer for sections (assign number to each section)
stringIndexer = StringIndexer(inputCol = 'section', outputCol = 'sectionIndex', handleInvalid = 'skip')

# tokenize abstracts (col with list of words)
tokenizer = Tokenizer(inputCol="combo", outputCol="words")

# remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# count vectorizer to return counts of words that appear in each abstract
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=100000)

# Decision Tree model to predict section based on words in abstract
dt = DecisionTreeClassifier(labelCol="sectionIndex", featuresCol="features")

# pipeline
dt_pipeline = Pipeline(stages = [stringIndexer, tokenizer, remover, countVectors, dt])

In [21]:
# training, testing split

seed = 42

(train_data, test_data) = df.randomSplit([0.8, 0.2], seed = seed)

In [22]:
# create model
dt_model = dt_pipeline.fit(train_data)

In [23]:
# make predictions
dt_preds = dt_model.transform(test_data)

In [24]:
# evaluate model on test data
ev = MulticlassClassificationEvaluator(labelCol = 'sectionIndex', predictionCol="prediction")

# f1
f1 = ev.evaluate(dt_preds)

# log loss
log_loss = ev.evaluate(dt_preds, {ev.metricName: "logLoss"})

# accuracy
accuracy = ev.evaluate(dt_preds, {ev.metricName: "accuracy"})

print("f1:", f1, "\nlog_loss:", log_loss, "\naccuracy:", accuracy)

f1: 0.15947392954873796 
log_loss: 2.8526123880697627 
accuracy: 0.22


In [25]:
# training accuracy
train_dt = dt_model.transform(train_data)

ev.evaluate(train_dt, {ev.metricName: "accuracy"})

0.20688121014385288

In [26]:
# from pyspark.sql.functions import monotonically_increasing_id, array, arrays_zip, explode
from pyspark.ml.functions import vector_to_array
from pyspark.sql.window import Window

### **Top K Accuracy**

In [27]:
# compute top k = 4 accuracy for predictions (i.e., correct section in the 5 classes with the highest probability)
top_4_accuracy = top_k_accuracy(dt_preds, 'section', 4)
top_4_accuracy

0.4772727272727273

In [28]:
# compute top k = 5 accuracy for predictions (i.e., correct section in the 5 classes with the highest probability)
top_5_accuracy = top_k_accuracy(dt_preds, 'section', 5)
top_5_accuracy

0.5312121212121212

#### **Attempt with `new_section` column instead (with new "Other" label)**

In [29]:
# new StringIndexer instance using the new_section column
stringIndexer_2 = StringIndexer(inputCol = 'new_section', outputCol = 'sectionIndex')

In [30]:
dt_pipeline_2 = Pipeline(stages = [stringIndexer_2, tokenizer, remover, countVectors, dt])

In [31]:
dt_model_2 = dt_pipeline_2.fit(train_data)

In [32]:
dt_preds_2 = dt_model_2.transform(test_data)

In [33]:
# f1
f1_2 = ev.evaluate(dt_preds_2)

# log loss
log_loss_2 = ev.evaluate(dt_preds_2, {ev.metricName: "logLoss"})

# accuracy
accuracy_2 = ev.evaluate(dt_preds_2, {ev.metricName: "accuracy"})

print("f1:", f1_2, "\nlog_loss:", log_loss_2, "\naccuracy:", accuracy_2)

f1: 0.15941412533723548 
log_loss: 2.796783526042492 
accuracy: 0.21993335352923357


#### **Create A Balanced Dataset to (Potentially) Improve Accuracy $\rightarrow$ Due to Highly Imbalanced Number of Articles Per Section**

In [34]:
import numpy as np

In [35]:
# balance the data (see ds5100_proj_funcs.py for more information) --> each section sampled so ~ mean num of articles in each
balanced_data = balance_categories(df, "new_section")

In [36]:
balanced_data.groupby('new_section').count().sort('count', ascending = False).show(26)

+--------------------+-----+
|         new_section|count|
+--------------------+-----+
|             Theater|  678|
|               World|  673|
|                Arts|  669|
|             Opinion|  666|
|        Business Day|  666|
|              Health|  659|
|          Technology|  655|
|              Travel|  654|
|                U.S.|  654|
|                Food|  653|
|            Magazine|  653|
|              Sports|  652|
|               Style|  652|
|               Books|  649|
|  Crosswords & Games|  646|
|              Movies|  645|
|               Other|  642|
|            New York|  642|
|       Times Insider|  639|
|                Well|  638|
|         Real Estate|  635|
|             Climate|  634|
|             Science|  633|
|The Learning Network|  633|
|            Podcasts|  628|
|          The Upshot|  622|
+--------------------+-----+



In [37]:
# generate balanced training and testing datasets

seed = 42

(train_balanced, test_balanced) = balanced_data.randomSplit([0.8, 0.2], seed = seed)

In [38]:
dt_model_3 = dt_pipeline_2.fit(train_balanced)

In [39]:
dt_preds_3 = dt_model_3.transform(test_balanced)

In [40]:
# f1
f1_3 = ev.evaluate(dt_preds_3)

# log loss
log_loss_3 = ev.evaluate(dt_preds_3, {ev.metricName: "logLoss"})

# accuracy
accuracy_3 = ev.evaluate(dt_preds_3, {ev.metricName: "accuracy"})

print("f1:", f1_3, "\nlog_loss:", log_loss_3, "\naccuracy:", accuracy_3)

f1: 0.10979513784154601 
log_loss: 3.026206315160336 
accuracy: 0.11280575539568345


In [41]:
# top k accuracy, k = 2
top_2_balanced = top_k_accuracy(dt_preds_3, 'section', 2)

top_2_balanced

0.1562589928057554

In [42]:
# top k accuracy, k = 5
top_5_balanced = top_k_accuracy(dt_preds_3, 'section', 5)

top_5_balanced

0.2725179856115108

#### **Sources**

* Read a CSV file into Spark: https://sparkbyexamples.com/pyspark/pyspark-read-csv-file-into-dataframe/
* Convert columns to different data type with `withColumn` and `df.col1.cast("dtype")`: https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/
* Check for missing values: from `data_preprocessing.ipydt` in `assingments/M4_3_data_preprocs_ungraded`)
* Extract single values from a dataframe in pyspark: https://www.geeksforgeeks.org/pyspark-extracting-single-value-from-dataframe/
* Drop multiple columns with `*`: https://sparkbyexamples.com/pyspark/pyspark-drop-column-from-dataframe/
* Argument `subset` for subset of columns when replacing null values with `df.fillna`: https://sparkbyexamples.com/pyspark/pyspark-fillna-fill-replace-null-values/
* Calculating percentage of total count for groupBy using pyspark: https://stackoverflow.com/questions/52283751/calculating-percentage-of-total-count-for-groupby-using-pyspark
* Spark Dataframe: How to add an index Column with `monotonically_increasing_id`: https://stackoverflow.com/questions/43406887/spark-dataframe-how-to-add-a-index-column-aka-distributed-data-index
* Classification and Regression in Spark: https://spark.apache.org/docs/latest/ml-classification-regression.html
* Multi-Class Text Classification with PySpark $\rightarrow$ for using `CountVectorizer`: https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35
* `CountVectorizer` documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.CountVectorizer.html#pyspark.ml.feature.CountVectorizer.minDF
* PySpark: modify column values when another column value satisfies a condition with `when` and `otherwise` from `pyspark.sql.functions`: https://stackoverflow.com/questions/43988801/pyspark-modify-column-values-when-another-column-value-satisfies-a-condition
* Get value of a particular cell in PySpark Dataframe: https://www.geeksforgeeks.org/get-value-of-a-particular-cell-in-pyspark-dataframe/
* Using a fraction > 1.0 for up-sampling data: https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark
* Append data to an empty dataframe in PySpark: https://www.geeksforgeeks.org/append-data-to-an-empty-dataframe-in-pyspark/
* `pyspark.sql.functions.explode` documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode.html
* `pyspark.sql.functions.arrays_zip` documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.arrays_zip.html
* Retrieve top n in each group of a DataFrame in pyspark: https://stackoverflow.com/questions/38397796/retrieve-top-n-in-each-group-of-a-dataframe-in-pyspark
* Introducing Window Functions in Spark SQL: https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html
* Convert PySpark DenseVector to array: https://stackoverflow.com/questions/58490770/convert-pyspark-densevector-to-array
* Aggregate function `collect_list`: https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/
* pyspark `collect_set` or `collect_list` with `groupby`: https://stackoverflow.com/questions/37580782/pyspark-collect-set-or-collect-list-with-groupby
* PySpark Join Types: https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
* Removing duplicate columns after join in PySpark by specificying duplicate column in join function with argument `['col1']`: https://www.geeksforgeeks.org/removing-duplicate-columns-after-dataframe-join-in-pyspark/#:~:text=Removing%20duplicate%20columns%20after%20join,and%20then%20drop%20duplicate%20columns
* Count categories in another column after `groupby` with `.agg(countDistict('col1'))`: https://stackoverflow.com/questions/46421677/how-to-count-unique-id-after-groupby-in-pyspark
* Checking dataframe equality with `df1.subtract(df2)`: https://community.databricks.com/s/question/0D53f00001HKHZeCAP/comparing-two-dataframes
* Cross validation in pyspark: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html?highlight=crossvalidator
* Get (default) metric of `MulticlassMetrics` evaluator with `getMetricName()` method: https://spark.apache.org/docs/1.6.1/api/java/org/apache/spark/ml/evaluation/MulticlassClassificationEvaluator.html
* Increasing driver memory in Jupyter: https://stackoverflow.com/questions/51601478/setting-pyspark-executor-memory-and-executor-core-within-jupyter-notebook
* Getting the best parameter combination from cross validation using `zip` with `.avgMetrics` and `paramGrid`: https://community.databricks.com/s/question/0D53f00001HKHh5CAH/pyspark-how-to-get-best-params-in-grid-search
* Visualizations (bar graphs) code: https://jkropko.github.io/surfing-the-data-pipeline/ch11.html#distributional-graphs
* Nested list comprehensions: https://www.geeksforgeeks.org/nested-list-comprehensions-in-python/
* Remove a legend from a plot in `seaborn`: https://www.delftstack.com/howto/seaborn/remove-legend-seaborn-plot/
* Get the value in a particular row and column in Spark: https://www.geeksforgeeks.org/get-value-of-a-particular-cell-in-pyspark-dataframe/
* `15. ML Model Selection and Tuning.ipynb`
* `11. ML Pipelines.ipynb`
* `10. ML Feature Utilities.ipynb`
* `12. MLlib Regression.ipynb`
* `08. MLlib Statistics.ipynb`
* `09. MLlib Classification.ipynb`
* `07. Machine Learning with MLlib - Overview.ipynb`
* `05. SparkSession, Spark SQL and DataFrames.ipynb`
* `04. Running Spark on a Cluster.ipynb`
* `03. KeyValuePairs.ipynb`
* `02. RDDs.ipynb`
* `01. SparkGettingStarted.ipynb`